<h1 style="color:rgb(17, 116, 155);text-align:left;font-size:250%;font-family:verdana;text-decoration:underline;"> 
    Advanced topics - Final Project - Part 3: End-to-End CNN</h1>

## <u>Imports</u>

In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

from PIL import Image, ImageFilter
import tensorflow as tf
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time
import torch.nn.functional as F
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import gc

In [4]:
# קביעת התקן (GPU אם קיים)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## <u>Load and Preprocess Dataset</u>

In [6]:
# טעינת הנתונים (TFRecord) והכנה לפורמט PyTorch
print("Loading Cars196 dataset from TFRecord files...")

data_dir = r"C:\Users\yifat\Data Science\נושאים מתקדמים\Final_Project_Shay\cars196"

# יצירת רשימת קבצי ה-TFRecord
train_files = [os.path.join(data_dir, f"cars196-train.tfrecord-0000{i}-of-00008") for i in range(8)]
test_files = [os.path.join(data_dir, f"cars196-test.tfrecord-0000{i}-of-00008") for i in range(8)]

# פונקציה לקריאת TFRecord
def parse_tfrecord(example):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    parsed_example = tf.io.parse_single_example(example, feature_description)
    
    image = tf.image.decode_jpeg(parsed_example['image'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # מונע איבוד מידע
    image = tf.image.resize(image, (224, 224))

    label = parsed_example['label']
    return image, label


def load_tfrecord_dataset(filenames):
    raw_dataset = tf.data.TFRecordDataset(filenames)
    dataset = raw_dataset.map(parse_tfrecord)
    return list(dataset)  # ממירים לרשימה לשימוש ב-PyTorch

# טעינת ה-Train/Test מ-TFRecord
train_data = [(image, label) for image, label in load_tfrecord_dataset(train_files)]
test_data = [(image, label) for image, label in load_tfrecord_dataset(test_files)]


print(f"Loaded {len(train_data)} training images and {len(test_data)} test images.")


Loading Cars196 dataset from TFRecord files...
Loaded 8144 training images and 8041 test images.


In [14]:
# מחלקת Dataset לטעינת הנתונים
class Cars196Dataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_tf, label = self.data[idx]  # image_tf הוא Tensor של TensorFlow
        image_np = image_tf.numpy()  # המרה ל-NumPy
        image = Image.fromarray((image_np * 255).astype(np.uint8))  # הבאת הפיקסלים לטווח 0-255
        if self.transform:
            image = self.transform(image)
        return image, label.numpy()  # גם label צריך להיות מספר ולא Tensor


In [16]:
# ניסיון עם אוגמנטציה פחות אגרסיבית
# הפחתת רמת ה-ColorJitter והאוגמנטציות הגורמות לעיוותים גדולים כמו RandomPerspective ו-RandomAffine
# הימנעות מיותר מדי RandomErasing, שכן זה יכול לגרום לאיבוד מידע חשוב
############################################################

In [18]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(p=0.5),  # היפוך אופקי
    transforms.RandomRotation(10),  # הקטנת סיבוב ל-10 מעלות
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),  # הקטנת שינויי הצבעים
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # פחות חיתוך
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# סט הבדיקה ללא אוגמנטציה
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [20]:
batch_size = 32
train_dataset = Cars196Dataset(train_data, transform=train_transform)
test_dataset = Cars196Dataset(test_data, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train and Test Dataloaders created with batch size {batch_size}.")
print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")


Train and Test Dataloaders created with batch size 32.
Training samples: 8144, Testing samples: 8041


In [22]:
# חישוב משקלים למחלקות לטיפול בחוסר איזון
labels_list = np.array([label.numpy() for _, label in train_data])  # המרה ל-NumPy
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels_list), y=labels_list)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)  # המרה ל-Tensor של PyTorch


## <u>Define and Train a CNN Model</u>

In [24]:
# הוספת שכבות קונבולוציה כדי להגדיל את עומק הרשת
# Batch Normalization לאחר כל שכבת קונבולוציה כדי לסייע ללמידה יציבה יותר
# הגדלת מספר הנוירונים בשכבות ה-FC

In [27]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes=196):
        super(CNNClassifier, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)  # הוספת נורמליזציה
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(512 * 14 * 14, 2048)  # הגדלת מספר הנוירונים
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


In [29]:
# יצירת המודל
model = CNNClassifier(num_classes=196).to(device)

# הגדרת פונקציית הפסד ואופטימיזציה
num_epochs = 50  # הגדלת מספר האפוקים

# פונקציית הפסד: שימוש בשקלול מחלקות בלבד
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)  # שינוי מ-SGD ל-AdamW
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.005, steps_per_epoch=len(train_loader), epochs=num_epochs)


In [30]:
torch.cuda.empty_cache()  # אם יש GPU 
gc.collect()  # ניקוי זיכרון בפייתון


537

In [33]:
# פונקציות אימון ובדיקה
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    model.to(device)
    running_loss = 0
    start_time = time.time()
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()  # עדכון הלרנינג רייט בסוף כל אפוק
    end_time = time.time()
    running_loss /= len(train_loader)
    return running_loss, end_time - start_time
    

# פונקציה לבדיקה של המודל
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        model.to(device)
        running_loss = 0
        total_predictions = 0
        correct_predictions = 0
        
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
        
        running_loss /= len(test_loader)
        accuracy = (correct_predictions / total_predictions)*100.0
        return running_loss, accuracy


In [35]:
# שילוב Early Stopping 
######################

In [37]:
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001):
        """
        patience - מספר האפוקים ללא שיפור לפני עצירה
        min_delta - השיפור המינימלי שייחשב לשיפור אמיתי
        """
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0  # אפס את הספירה אם יש שיפור
        else:
            self.counter += 1  # ספירה של אפוקים ללא שיפור

        return self.counter >= self.patience  # עצור אם חרגנו ממספר האפוקים המותר


In [39]:
# אימון הרשת
early_stopping = EarlyStopping(patience=10, min_delta=0.001)  # עצירה אחרי 10 אפוקים ללא שיפור

num_epochs = 50  # הגדלת מספר האפוקים

for epoch in range(num_epochs):
    train_loss, train_time = train_epoch(model, train_loader, criterion, optimizer)
    test_loss, test_acc = test_model(model, test_loader, criterion)

    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss = {train_loss:.6f}, Time = {train_time:.2f}s")
    print(f"Test Loss = {test_loss:.6f}, Accuracy = {test_acc:.2f}%")
    print('-' * 60)

    if early_stopping(test_loss):  # בדיקה אם צריך לעצור
        print(f"Early stopping at epoch {epoch+1}")
        break


# שמירת המודל
torch.save(model.state_dict(), "cars196_cnn_trained.pth")
print("End-to-End CNN model training completed and saved!")


Epoch 1/50: Train Loss = 5.559738, Time = 407.28s
Test Loss = 5.216059, Accuracy = 1.03%
------------------------------------------------------------
Epoch 2/50: Train Loss = 5.222404, Time = 393.56s
Test Loss = 5.192390, Accuracy = 0.99%
------------------------------------------------------------
Epoch 3/50: Train Loss = 5.198333, Time = 392.68s
Test Loss = 5.185934, Accuracy = 1.02%
------------------------------------------------------------
Epoch 4/50: Train Loss = 5.190670, Time = 411.49s
Test Loss = 5.172846, Accuracy = 0.81%
------------------------------------------------------------
Epoch 5/50: Train Loss = 5.169718, Time = 408.84s
Test Loss = 5.170104, Accuracy = 1.07%
------------------------------------------------------------
Epoch 6/50: Train Loss = 5.158999, Time = 410.17s
Test Loss = 5.130618, Accuracy = 1.39%
------------------------------------------------------------
Epoch 7/50: Train Loss = 5.145087, Time = 411.32s
Test Loss = 5.126859, Accuracy = 1.21%
-----------